
# Storage and volumes

## Bind mounts

Consider the example in figure 4.3. Suppose you’re running a web server that
depends on sensitive configuration on the host and emits access logs that need to be
forwarded by your log-shipping system. You could use Docker to launch the web
server in a container and bind-mount the configuration location as well as the location
where you want the web server to write logs.

You can try this for yourself. Create a placeholder log file and create a special NGINX
configuration file named example.conf. Run the following commands to create and
populate the files:

`gradiva/vaja01`

    touch ./example.log
    cat >./example.conf <<EOF
    server {
        listen 80;
        server_name localhost;
        access_log /var/log/nginx/custom.host.access.log main;
        location / {
            root /usr/share/nginx/html;
            index index.html index.htm;
        }
    }
    EOF

Once a server is started with this configuration file, it will offer the NGINX default site
at `http://localhost/`, and access logs for that site will be written to a file in the container
at `/var/log/nginx/custom.host.access.log`. The following command will start
an NGINX HTTP server in a container where your new configuration is bind-mounted
to the server’s configuration root:

    MAIN_PATH=${PWD}/gradiva/vaja01; \
    CONF_SRC=${MAIN_PATH}/example.conf; \
    CONF_DST=/etc/nginx/conf.d/default.conf; \
    LOG_SRC=${MAIN_PATH}/example.log; \
    LOG_DST=/var/log/nginx/custom.host.access.log; \
    docker run -d --name diaweb \
        --mount type=bind,src=${CONF_SRC},dst=${CONF_DST} \
        --mount type=bind,src=${LOG_SRC},dst=${LOG_DST} \
        -p 80:80 \
        nginx:latest

With this container running, you should be able to point your web browser at
`http://localhost/` and see the NGINX hello-world page, and you will not see any
access logs in the container log stream: docker logs diaweb. However, you will be
able to see those logs if you examine the example.log file in your home directory:
`cat ./example.log`.

This example touches on an important feature of volumes. When you mount a volume
on a container filesystem, it replaces the content that the image provides at that
location. By default, the `nginx:latest` image provides some default configuration at
`/etc/nginx/conf.d/default.conf`, but when you created the bind mount with a destination
at that path, the content provided by the image was overridden by the content
on the host. This behavior is the basis for the polymorphic container pattern discussed
later in the chapter.

Expanding on this use case, suppose you want to make sure that the NGINX web
server can’t change the contents of the configuration volume. Even the most trusted
software can contain vulnerabilities, and it’s best to minimize the impact of an attack on
your website. Fortunately, Linux provides a mechanism to make mount points read-only.
You can do this by adding the `readonly=true` argument to the mount specification. In
the example, you should change the run command to something like the following:

    docker rm -f diaweb

    MAIN_PATH=/home/leon11/docker-k8s/docker/Storage_and_volumes/gradiva/vaja01; \
    CONF_SRC=${MAIN_PATH}/example.conf; \
        CONF_DST=/etc/nginx/conf.d/default.conf; \
        LOG_SRC=${MAIN_PATH}/example.log; \
    LOG_DST=/var/log/nginx/custom.host.access.log; \
    docker run -d --name diaweb \
        --mount type=bind,src=${CONF_SRC},dst=${CONF_DST},readonly=true \
        --mount type=bind,src=${LOG_SRC},dst=${LOG_DST} \
        -p 80:80 \
        nginx:latest

By creating the read-only mount, you can prevent any process inside the container from
modifying the content of the volume. You can see this in action by running a quick test:

    docker exec diaweb \
    sed -i "s/listen 80/listen 8080/" /etc/nginx/conf.d/default.conf

This command executes a sed command inside the diaweb container and attempts
to modify the configuration file. The command fails because the file is mounted as
read-only.

The first problem with bind mounts is that they tie otherwise portable container
descriptions to the filesystem of a specific host. If a container description depends on
content at a specific location on the host filesystem, that description isn’t portable to
hosts where the content is unavailable or available in some other location.

The next big problem is that they create an opportunity for conflict with other
containers. It would be a bad idea to start multiple instances of Cassandra that all use
the same host location as a bind mount for data storage. In that case, each of the
instances would compete for the same set of files. Without other tools such as file
locks, that would likely result in corruption of the database.

Bind mounts are appropriate tools for workstations, machines with specialized
concerns, or in systems combined with more traditional configuration management
tooling. It’s better to avoid these kinds of specific bindings in generalized platforms or
hardware pools.